# Agentic Evaluation (ReAct-style)

This notebook adds a minimal ReAct-style loop for NL→SQL. It reuses the same benchmark (`data/classicmodels_test_200.json`) and metrics (VA/EX/EM; TS planned) to measure gains over prompt-only and QLoRA runs.

Plan (step-by-step):
1) Clone repo (Colab) + install deps
2) Environment + DB connection
3) Load schema summary + test set
4) Load model (base or QLoRA adapters)
5) Define ReAct prompt + loop (Thought → Action → Observation → Refinement)
6) Run evaluation (VA/EX/EM) and save to `results/agent/…`


In [ ]:
# 0) Clone repo (Colab) + install deps
import os
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    if not os.path.exists('/content/NLtoSQL'):
        !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git /content/NLtoSQL
    %cd /content/NLtoSQL
    !pip -q install -r requirements.txt
    import torch, transformers, accelerate, peft
    print('torch', torch.__version__, 'cuda', torch.cuda.is_available())
else:
    print('Not in Colab; using existing workspace')


## Optional: use gcloud ADC (without a key)
If you don't have a service-account JSON handy in Colab, authenticate once per session:

```bash
!gcloud auth application-default login
```

Then set the DB env vars and run the connection cell below.

In [ ]:
# Run this only if you prefer gcloud-based ADC (no JSON key)
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    %pip install -q --upgrade google-auth google-auth-oauthlib
    !gcloud auth application-default login
else:
    print("Not in Colab; skip gcloud auth.")


## Runtime setup (pinned CUDA/torch/bnb stack)
Run once per fresh Colab session (after selecting a GPU). This pins torch + bitsandbytes + triton to versions known to work with 4-bit QLoRA. After it finishes, restart the runtime and rerun from the top.

In [ ]:
%%bash
# Torch + CUDA 12.1
pip install -q --force-reinstall --no-cache-dir   torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121   --index-url https://download.pytorch.org/whl/cu121

# bitsandbytes + triton + HF stack pins
pip install -q --force-reinstall --no-cache-dir   bitsandbytes==0.43.3 triton==2.3.1   transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0


In [ ]:

# 1) Environment + DB
import os
from getpass import getpass
from pathlib import Path

from google.cloud.sql.connector import Connector
from google.oauth2.service_account import Credentials
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine

# Expected env vars (set these in a Colab cell):
# GOOGLE_APPLICATION_CREDENTIALS=/content/sa.json
# INSTANCE_CONNECTION_NAME, DB_USER, DB_PASS, DB_NAME
INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME") or "classicmodels"
GOOGLE_CREDS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

creds = None
if GOOGLE_CREDS:
    creds = Credentials.from_service_account_file(GOOGLE_CREDS)
    print(f"Using service account from {GOOGLE_CREDS}")
else:
    print("Using default ADC (gcloud auth or Colab auth). If this fails, set GOOGLE_APPLICATION_CREDENTIALS.")

connector = Connector(credentials=creds)

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
    )

engine: Engine = create_engine(
    "mysql+pymysql://",
    creator=getconn,
    future=True,
)

with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("DB connection OK")


In [ ]:
# 2) Load schema summary + test set
import json
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME)

test_path = Path("data/classicmodels_test_200.json")
test_set = json.loads(test_path.read_text(encoding="utf-8"))
print("Test items:", len(test_set))


In [ ]:

# 3) Load model (base or QLoRA adapters)
import os
from getpass import getpass
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
ADAPTER_PATH = "results/adapters/qlora_classicmodels"  # set to None to use base model

HF_TOKEN = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_TOKEN")
if not HF_TOKEN:
    HF_TOKEN = getpass("Enter HF_TOKEN (https://huggingface.co/settings/tokens): ").strip()

cc_major, cc_minor = torch.cuda.get_device_capability(0) if torch.cuda.is_available() else (0, 0)
use_bf16 = cc_major >= 8
compute_dtype = torch.bfloat16 if use_bf16 else torch.float16
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
print("Using bf16:", use_bf16)

# Tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Quantized base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map={"": 0} if torch.cuda.is_available() else None,
    token=HF_TOKEN,
)
base_model.generation_config.do_sample = False
base_model.generation_config.temperature = 1.0
base_model.generation_config.top_p = 1.0

if ADAPTER_PATH:
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH, token=HF_TOKEN)
    print("Loaded adapters from", ADAPTER_PATH)
else:
    model = base_model
    print("Using base model (no adapters)")


In [ ]:
# 4) ReAct helpers: prompt builder and loop
import re
from nl2sql.query_runner import QueryRunner

runner = QueryRunner(engine)

def build_react_prompt(nlq: str, schema_text: str, history: list, observation: str) -> str:
    history_text = "\n".join([
        f"Thought/Action: {h['ta']}\nObservation: {h['obs']}" for h in history
    ])
    return f"""
You are an expert SQL agent. Follow the steps:
1) Think briefly about the question.
2) Propose a single SQL query.
3) If previous Observation reports an ERROR, fix it.

Schema:
{schema_text}

Question: {nlq}

Previous trace:
{history_text}
Observation: {observation}

Now reply with SQL only.
"""

def extract_sql(text: str) -> str:
    m = re.search(r"(SELECT[\s\S]+?);", text, re.IGNORECASE)
    return m.group(1) + ";" if m else text.strip()

def react_sql(nlq: str, schema_text: str, max_steps: int = 3):
    history = []
    observation = "Start."
    final_sql = None
    for step in range(max_steps):
        prompt = build_react_prompt(nlq, schema_text, history, observation)
        inputs = tok(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=192)
        gen = tok.decode(out[0], skip_special_tokens=True)
        sql = extract_sql(gen)
        try:
            runner.run(sql)
            observation = "SUCCESS"
            final_sql = sql
            break
        except Exception as e:
            observation = f"ERROR: {e}"
        history.append({"ta": gen, "obs": observation})
    return final_sql or sql, history


In [ ]:
# 5) Evaluation loop (VA/EX/EM). TS is planned.
from nl2sql.eval import execution_accuracy

results = []
for i, sample in enumerate(test_set, start=1):
    nlq = sample["nlq"]
    gold_sql = sample["sql"]
    pred_sql, trace = react_sql(nlq, SCHEMA_SUMMARY, max_steps=3)
    va = 0
    em = int(pred_sql.strip().rstrip(";").lower() == gold_sql.strip().rstrip(";").lower())
    ex = 0
    try:
        runner.run(pred_sql)
        va = 1
        ex = int(execution_accuracy(engine, pred_sql, gold_sql))
    except Exception:
        va = 0
        ex = 0
    results.append({
        "nlq": nlq,
        "gold_sql": gold_sql,
        "pred_sql": pred_sql,
        "va": va,
        "em": em,
        "ex": ex,
        "trace": trace,
    })
    if i % 20 == 0:
        print(f"Processed {i}/{len(test_set)}")

va_rate = sum(r["va"] for r in results) / len(results)
ex_rate = sum(r["ex"] for r in results) / len(results)
em_rate = sum(r["em"] for r in results) / len(results)
print("VA:", va_rate, "EX:", ex_rate, "EM:", em_rate)

Path("results/agent").mkdir(parents=True, exist_ok=True)
save_path = Path("results/agent/results_react_200.json")
save_path.write_text(json.dumps({
    "va_rate": va_rate,
    "ex_rate": ex_rate,
    "em_rate": em_rate,
    "items": results,
}, ensure_ascii=False, indent=2), encoding="utf-8")
print("Saved to", save_path)
